In [1]:
import pickle
import numpy as np
import pandas as pd
import polars as pl

In [2]:
year = 2023
month = 3
taxi_type = 'yellow'

input_file = f"https://d37ci6vzurychx.cloudfront.net/trip-data/{taxi_type}_tripdata_{year:04d}-{month:02d}.parquet"
output_file = f"../outputs/{taxi_type}_tripdata_{year:04d}-{month:02d}_prediction.parquet"

saved_model = "../models/model.bin" 

In [3]:
# Load the model
def load_model(saved_model):
    with open(saved_model, 'rb') as f_in:
        dv, model = pickle.load(f_in)
    return dv, model


In [4]:
def read_dataframe(filename, categorical):
    df = pl.scan_parquet(filename)

    df = df.with_columns(
        duration=(pl.col('tpep_dropoff_datetime') - pl.col('tpep_pickup_datetime')).dt.total_seconds() / 60
    ).filter(
        (pl.col('duration') >= 1) & (pl.col('duration') <= 60)
    ).with_columns(
        pl.col('PULocationID').fill_null(value=-1).cast(pl.Int16).cast(pl.Utf8).alias('PULocationID'),
        pl.col('DOLocationID').fill_null(value=-1).cast(pl.Int16).cast(pl.Utf8).alias('DOLocationID')
    )

    df = df.select(categorical+['duration'])

    return df.collect().to_pandas()


In [5]:
def prepare_dictionaries(df, categorical):
    return df[categorical].to_dict(orient='records')


In [6]:
def apply_model(saved_model, input_file, output_file):
    categorical = ['PULocationID', 'DOLocationID']
    df = read_dataframe(input_file, categorical)
    dicts = prepare_dictionaries(df, categorical)

    dv, model = load_model(saved_model)

    X = dv.transform(dicts)
    y_pred = model.predict(X)

    df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')
    
    df_result = pd.DataFrame()
    df_result['ride_id'] = df['ride_id']
    df_result['predicted_duration'] = y_pred

    df_result.to_parquet(
        output_file,
        engine='pyarrow',
        compression=None,
        index=False
    )


In [7]:
apply_model(saved_model, input_file, output_file)